In [1]:
import pennylane as qml

In [2]:
## Include other required notebooks.
%run prepareVQE.ipynb

In [3]:
## Define the net charge of BeH2.
BeHHcharge = 0

In [4]:
# Restrict active space
active_electrons = 4
active_orbitals = 6

h_active, (n_qubits_active, n_electrons) = genBeH2(genSTO3Gopt, name="BeH2_quiqbox", 
    active_electrons=active_electrons, active_orbitals=active_orbitals, grouping_type="qwc"
)
h_active_sparse = qml.utils.sparse_hamiltonian(h_active)

In [5]:
print("\n<Info of Adapt-VQE with selected active space>")
print("Number of qubits needed for the selected active space:", n_qubits_active)
print('Number of Pauli strings (with grouping):', len(h_active.grouping_indices))


<Info of Adapt-VQE with selected active space>
Number of qubits needed for the selected active space: 12
Number of Pauli strings (with grouping): 99


In [6]:
hf_state_active = qml.qchem.hf_state(active_electrons,n_qubits_active)
singles_active, doubles_active = qml.qchem.excitations(active_electrons,n_qubits_active)

In [7]:
def adapt_circuit_1_active(params, excitations):
    qml.BasisState(hf_state_active, wires=range(n_qubits_active))

    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=excitation)
        else:
            qml.SingleExcitation(params[i], wires=excitation)
    return qml.expval(qml.SparseHamiltonian(h_active_sparse, wires=h_active.wires))

In [8]:
# circuit used for fixing parameters of double excition gates
# and optimizing parameters of single excitation gates
def adapt_circuit_2_active(params, excitations, gates_select, params_select):
    qml.BasisState(hf_state_active, wires=range(n_qubits_active))

    for i, gate in enumerate(gates_select):
        if len(gate) == 4:
            qml.DoubleExcitation(params_select[i], wires=gate)
        elif len(gate) == 2:
            qml.SingleExcitation(params_select[i], wires=gate)

    for i, gate in enumerate(excitations):
        if len(gate) == 4:
            qml.DoubleExcitation(params[i], wires=gate)
        elif len(gate) == 2:
            qml.SingleExcitation(params[i], wires=gate)
    return qml.expval(qml.SparseHamiltonian(h_active_sparse, wires=h_active.wires))

In [9]:
## Construct Adapt-VQE.
dev = qml.device("lightning.qubit", wires=h_active.wires)
diff_method = 'adjoint'
cost_fn_final_active, params_final_active, gates_select_active = constructAdaptVQE(dev, 
    diff_method, adapt_circuit_1_active, adapt_circuit_2_active, doubles_active, singles_active
)

Original Number of double excitation gates: 76
Number of double excitation gates after filtering: 14
Original Number of single excitation gates: 16
Number of single excitation gates after filtering: 4


In [10]:
## Run the constructed VQE.
## For excitations we'll use our singles and doubles selected gates, i.e., `gates_select`.
maxStep = 50
adamStep = 0.02
opt_adam = qml.AdamOptimizer(stepsize=adamStep, beta1=0.9, beta2=0.99, eps=1e-08)
adapt_energies_active, adapt_runtime_active = run_adapt_vqe(
    cost_fn_final_active, params_final_active, opt_adam, maxStep, excitations=gates_select_active
)

Completed iteration 5
Energy: -15.6196737456201 Ha
Step Time: 0.32550549507141113 s
----------------
Completed iteration 10
Energy: -15.618103964976491 Ha
Step Time: 0.32205915451049805 s
----------------
Completed iteration 15
Energy: -15.619762006045065 Ha
Step Time: 0.3365011215209961 s
----------------
Completed iteration 20
Energy: -15.620608316086818 Ha
Step Time: 0.779543399810791 s
----------------
Completed iteration 25
Energy: -15.620744541197467 Ha
Step Time: 0.3598930835723877 s
----------------
Completed iteration 30
Energy: -15.62096108324566 Ha
Step Time: 0.40140533447265625 s
----------------
Completed iteration 35
Energy: -15.621155101334304 Ha
Step Time: 0.36940717697143555 s
----------------
Completed iteration 40
Energy: -15.621158139432707 Ha
Step Time: 0.3594322204589844 s
----------------
Completed iteration 45
Energy: -15.621242353563188 Ha
Step Time: 0.33476758003234863 s
----------------
Completed iteration 50
Energy: -15.621261409460399 Ha
Step Time: 0.356560